# Artificial Neural Network Methods

## Imports 

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from xgboost import XGBRegressor, plot_importance
import shap

sns.set_theme(style="whitegrid")

In [ ]:
import sys; sys.path.insert(0, '..')

from src import preprocess as dp
from src import models

In [ ]:
DEBUG = False
N_ASSETS = 1
EPOCHS = 200
WINDOW_SIZE = 15
BATCH_SIZE = 1024
PCT_VALIDATION = 10 # last 10% of the data are used as validation set

## Data Loading

Set the correct path for the data folder

In [ ]:
#data_path = '/kaggle/input/'
data_path = '../data/'

Import asset details

In [ ]:
asset_details = pd.read_csv(data_path + 'g-research-crypto-forecasting/asset_details.csv')

In [ ]:
asset_details.sort_values(by='Asset_ID')

Import training data

In [ ]:
train = pd.read_csv(data_path + "g-research-crypto-forecasting/train.csv")
train

In [ ]:
train = train[train.Asset_ID == 1].copy()

In [ ]:
# Convert timestamp
train['timestamp'] = pd.to_datetime(train['timestamp'], unit='s')
train

In [ ]:
# Smaller dataset for debugging
if DEBUG:
    train = train[train.timestamp.dt.year >= 2020].copy()

In [ ]:
train

## Data Pre-Processing

###  Feature Engineering

#### Robust Scaler

In [ ]:
train_robust = dp.process_all_assets(train, scaler=RobustScaler())

In [ ]:
train_robust.head()

In [ ]:
train_robust[~train_robust.is_real].shape

In [ ]:
feature_cols = train_robust.columns.drop(['Asset_ID', 'Target', 'timestamp', 'is_real', 'Open', 'Close', 'High', 'Low', 'Median', 'Mean', 'VWAP'])

In [ ]:
feature_cols

In [ ]:
len(feature_cols)

In [ ]:
model = XGBRegressor()
model.fit(train_robust[feature_cols], train_robust.Target)

In [ ]:
# Initialize the matplotlib figure
def importance_plot(importance_dict, gain_type):
    _gain_type = 'Gain' if gain_type.lower() == 'gain' else 'Total Gain' if gain_type.lower() == 'total_gain' else gain_type
    series = pd.Series(importance_dict).sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(5, 5))
    sns.set_color_codes("muted")
    sns.barplot(x=series, y=series.index, color="b").set_title(f'Feature Importance ({_gain_type})')
    ax.set(ylabel="Feature", xlabel=_gain_type)
    return fig

In [ ]:
fig = importance_plot(model.get_booster().get_score(importance_type='gain'), 'gain')
fig.savefig(fig.axes[0].title.get_text()+'_robust.svg', format="svg", bbox_inches = "tight")

In [ ]:
fig = importance_plot(model.get_booster().get_score(importance_type='total_gain'), 'total_gain')
fig.savefig(fig.axes[0].title.get_text()+'_robust.svg', format="svg", bbox_inches = "tight")

In [ ]:
def get_best_features(xgbr_model, gain_threshold, total_gain_threshold):
    feature_importance_total_gain = pd.Series(xgbr_model.get_booster().get_score(importance_type='total_gain')).sort_values(ascending=False)
    feature_importance_gain = pd.Series(xgbr_model.get_booster().get_score(importance_type='gain')).sort_values(ascending=False)
    tot_g_feat = feature_importance_total_gain[:10].index.tolist()
    g_feat = feature_importance_gain[:10].index.tolist()
    return list(set(g_feat).intersection(set(tot_g_feat)))

In [ ]:
feature_cols = get_best_features(model, gain_threshold=0.0004, total_gain_threshold=0.1)

In [ ]:
feature_cols = ['STOCHRSIk_14_14_3_3', 'DPO_20', 'Spread', 'MACDh_12_26_9', 'Mean_trade', 'MACDs_12_26_9', 'Volume', 'MACD_12_26_9']

In [ ]:
train_robust[feature_cols]

In [ ]:
len(feature_cols)

#### MinMax Scaler

In [ ]:
train_minmax = dp.process_all_assets(train, scaler=MinMaxScaler())

In [ ]:
train_minmax.head()

In [ ]:
train_minmax[~train_minmax.is_real].shape

In [ ]:
feature_cols_mm = train_minmax.columns.drop(['Asset_ID', 'Target', 'timestamp', 'is_real', 'Open', 'Close', 'High', 'Low', 'Median', 'Mean', 'VWAP'])

In [ ]:
feature_cols_mm

In [ ]:
len(feature_cols_mm)

In [ ]:
model = XGBRegressor()
model.fit(train_minmax[feature_cols_mm], train_minmax.Target)

In [ ]:
fig = importance_plot(model.get_booster().get_score(importance_type='gain'), 'gain')
fig.savefig(fig.axes[0].title.get_text()+'_mm.svg', format="svg", bbox_inches = "tight")

In [ ]:
fig = importance_plot(model.get_booster().get_score(importance_type='total_gain'), 'total_gain')
fig.savefig(fig.axes[0].title.get_text()+'_mm.svg', format="svg", bbox_inches = "tight")

In [ ]:
feature_cols_mm = get_best_features(model, gain_threshold=0.0004, total_gain_threshold=0.1)

In [ ]:
feature_cols_mm

In [ ]:
feature_cols_mm = ['Count', 'STOCHRSIk_14_14_3_3', 'DPO_20', 'Spread', 'Mean_trade', 'MACDs_12_26_9', 'MACD_12_26_9']

### Time Seties Reconstruction

In [ ]:
def train_val_batches(df, feature_cols):
    # Reshaping target
    targets = df['Target'].to_numpy().reshape(-1, N_ASSETS)

    # Reshaping trainign data
    train_data = df[feature_cols].values
    train_data = train_data.reshape(-1, N_ASSETS, train_data.shape[-1])

    # Train / Validation splitting

    X_train, X_test = train_data[:-len(train_data)//PCT_VALIDATION], train_data[-len(train_data)//PCT_VALIDATION:]
    print(len(X_train))
    y_train, y_test = targets[:-len(train_data)//PCT_VALIDATION], targets[-len(train_data)//PCT_VALIDATION:]

    # Batch generators
    train_generator = models.sample_generator(X_train, y_train, length=WINDOW_SIZE, batch_size=BATCH_SIZE)
    val_generator = models.sample_generator(X_test, y_test, length=WINDOW_SIZE, batch_size=BATCH_SIZE)

    return train_generator, val_generator, len(X_train), y_test

In [ ]:
train_generator, val_generator, X_train_len, y_test = train_val_batches(train_robust, feature_cols)
print(f'Sample shape: {train_generator[0][0].shape}')
print(f'Target shape: {train_generator[0][1].shape}')

In [ ]:
train_generator_mm, val_generator_mm, _, _ = train_val_batches(train_minmax, feature_cols_mm)
print(f'Sample shape: {train_generator_mm[0][0].shape}')
print(f'Target shape: {train_generator_mm[0][1].shape}')

In [ ]:
tf.random.set_seed(0)
estop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min', restore_best_weights=True)
scheduler = keras.optimizers.schedules.ExponentialDecay(1e-3, (0.5 * X_train_len / BATCH_SIZE), 1e-3)
lr = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

## LSTM

In [ ]:
performances_df = pd.DataFrame()

### Single-Layered Multivariate LSTM

MinMax Scaler

In [ ]:
model_1_mm = models.get_model_LSTM(train_generator_mm, N_ASSETS)
model_1_mm.summary()

In [ ]:
epochs = EPOCHS
history_1_mm = model_1_mm.fit(train_generator_mm, validation_data=(val_generator_mm), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_1_mm = models.plot_training_history(history_1_mm)
fig_1_mm.show()

In [ ]:
predictions_1_mm = model_1_mm.predict(val_generator_mm)

In [ ]:
model_1_mm_performance = models.prediction_details(predictions=predictions_1_mm,
                                                   y_test=y_test,
                                                   window_size=WINDOW_SIZE,
                                                   asset_details=asset_details,
                                                   model_name=model_1_mm.name + '_mm',
                                                   assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_1_mm_performance, ignore_index=True)
performances_df

Robust Scaler

In [ ]:
model_1 = models.get_model_LSTM(train_generator, N_ASSETS)
model_1.summary()

In [ ]:
epochs = EPOCHS
history_1 = model_1.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_1 = models.plot_training_history(history_1)
fig_1.show()

In [ ]:
predictions_1 = model_1.predict(val_generator)

In [ ]:
model_1_performance = models.prediction_details(predictions=predictions_1,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_1.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_1_performance, ignore_index=True)
performances_df

### Single-Layered Multivariate LSTM With Droupouts

In [ ]:
model_2 = models.get_model_LSTM_dropout(train_generator, N_ASSETS)
model_2.summary()

In [ ]:
epochs = EPOCHS
history_2 = model_2.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_2 = models.plot_training_history(history_2)
fig_2.show()

In [ ]:
predictions_2 = model_2.predict(val_generator)

In [ ]:
model_2_performance = models.prediction_details(predictions=predictions_2,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_2.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_2_performance, ignore_index=True)
performances_df

### Double-Layered Multivariate LSTM

In [ ]:
model_3 = models.get_model_Double_LSTM(train_generator, N_ASSETS)
model_3.summary()

In [ ]:
epochs = EPOCHS
history_3 = model_3.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_3 = models.plot_training_history(history_3)
fig_3.show()

In [ ]:
predictions_3 = model_3.predict(val_generator)

In [ ]:
model_3_performance = models.prediction_details(predictions=predictions_3,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_3.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_3_performance, ignore_index=True)
performances_df

### Triple-Layered Multivariate LSTM

In [ ]:
model_4 = models.get_model_Triple_LSTM(train_generator, N_ASSETS)
model_4.summary()

In [ ]:
epochs = EPOCHS
history_4 = model_4.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_4 = models.plot_training_history(history_4)
fig_4.show()

In [ ]:
predictions_4 = model_4.predict(val_generator)

In [ ]:
model_4_performance = models.prediction_details(predictions=predictions_4,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_4.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_4_performance, ignore_index=True)
performances_df

### Conv1D-LSTM

In [ ]:
model_5 = models.get_model_Conv1D_Double_LSTM(train_generator, N_ASSETS)
model_5.summary()

In [ ]:
epochs = EPOCHS
history_5 = model_5.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_5 = models.plot_training_history(history_5)
fig_5.show()

In [ ]:
predictions_5 = model_5.predict(val_generator)

In [ ]:
model_5_performance = models.prediction_details(predictions=predictions_5,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_5.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_5_performance, ignore_index=True)
performances_df

### Bidirectional Double-Layered Multivariate LSTM

In [ ]:
model_6 = models.get_model_Bidirectional_2_layer_LSTM(train_generator, N_ASSETS)
model_6.summary()

In [ ]:
epochs = EPOCHS
history_6 = model_6.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_6 = models.plot_training_history(history_6)
fig_6.show()

In [ ]:
predictions_6 = model_6.predict(val_generator)

In [ ]:
model_6_performance = models.prediction_details(predictions=predictions_6,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_6.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_6_performance, ignore_index=True)
performances_df

### TCN

In [ ]:
model_7 = models.get_model_TCN(train_generator, N_ASSETS)
model_7.summary()

In [ ]:
epochs = EPOCHS
history_7 = model_7.fit(train_generator, validation_data=(val_generator), epochs=epochs, callbacks=[lr, estop])

In [ ]:
fig_7 = models.plot_training_history(history_7)
fig_7.show()

In [ ]:
predictions_7 = model_7.predict(val_generator)

In [ ]:
model_7_performance = models.prediction_details(predictions=predictions_7,
                                                y_test=y_test,
                                                window_size=WINDOW_SIZE,
                                                asset_details=asset_details,
                                                model_name=model_7.name,
                                                assets=train.Asset_ID.unique())

In [ ]:
performances_df = performances_df.append(model_7_performance, ignore_index=True)
performances_df